In [1]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

import pandas as pd
import json
import ast

In [9]:
def get_area_name(loc):
    """
    This function gets the address of a (latitude, longitude) position and returns the name of the area
    """

    geolocator = Nominatim(user_agent="geoapiExercises") 
    location = geolocator.reverse(loc) 

    location_address = location.raw['address']
    
    area_road = location_address.get('road', '')
    area_neighbourhood = location_address.get('neighbourhood', '')
    area_suburb = location_address.get('suburb', '')
    area_postcode = location_address.get('postcode', '')
    
    area_name = ""

    if area_road:
        area_name += area_road + ", "

    if area_neighbourhood:
        area_name += area_neighbourhood + ", "

    if area_suburb:
        area_name += area_suburb + ", "

    if area_postcode:
        area_name += area_postcode

    return area_name

In [5]:
with open('data/arrets.json') as f:

    data = json.load(f)
    
    df = pd.json_normalize(data)
    df = df.rename(columns={
        "fields.arrgeopoint": "start_station_coordinates",
        "fields.arrname": "start_station_name",
        "fields.arrtown": "station_town",
        "fields.arrtype": "station_type"
    })
    
    df = df[["start_station_coordinates", "start_station_name", "station_town", "station_type"]]

    df.to_csv('data/bus_stations.csv', index=False)

In [4]:
df.head()

,start_station_coordinates,start_station_name,station_town
0,"[49.04640369689759, 2.0932062985632944]",Gare de Pontoise,Pontoise
1,"[49.014610143868886, 2.0794749770046517]",Gare de Neuville Université,Neuville-sur-Oise
2,"[49.0449454931768, 2.10741709344313]",Gare de Mairie St-Ouen-l'Aumône,Saint-Ouen-l'Aumône
3,"[49.039423818329375, 2.0764361857497593]",Gare de Cergy-Préfecture,Cergy
4,"[49.03432274959586, 2.081983828653887]",Gare de Cergy-Préfecture,Cergy


In [37]:
df['station_type_voie'].unique()

array(['VDS', 'VSU1', 'VSU2', 'VDSC'], dtype=object)

In [38]:
df['station_dep'].unique()

array(['77', '78', '92', '95', '91', '94', '75', '93'], dtype=object)

In [39]:
df['line_separation'].unique()

array(['PR', 'FR'], dtype=object)

In [25]:
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculates the distance between two sets of latitude and longitude coordinates.
    """

    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

In [40]:
def find_nearest_bus_station(start_lat, start_long, dest_lat, dest_long):
    """
    Finds the nearest station with a type RER, TRAIN, METRO, TRAMWAY or VAL to the given GPS coordinates
    """

    # Get station information
    bus_stations = pd.read_csv("data/bus_stations.csv")

    # Converts the station_coordinates from type object to type list
    bus_stations['station_coordinates'] = bus_stations['station_coordinates'].apply(ast.literal_eval)

    # Create a column with the distance between each station and the input GPS coordinates
    bus_stations['distance_from_input'] = bus_stations.apply(
        lambda row: calculate_distance(start_lat, start_long, row['station_coordinates'][0], row['station_coordinates'][1]),
        axis=1
    )

    # Create a column with the distance between each station and the destination GPS coordinates
    bus_stations['distance_from_destination'] = bus_stations.apply(
        lambda row: calculate_distance(dest_lat, dest_long, row['station_coordinates'][0], row['station_coordinates'][1]),
        axis=1
    )

    # Sort stations by destination distance in ascending order
    sorted_dest_stations = bus_stations.sort_values(by='distance_from_destination')

    # Step 1: Get the closest stations to the destination
    closest_dest_station = sorted_dest_stations.iloc[0]
    closest_dest_station_coords = (closest_dest_station['station_coordinates'][0], closest_dest_station['station_coordinates'][1])
    closest_dest_station_type = closest_dest_station['station_type_voie']

    # Filter stations with station_type 
    filtered_stations = bus_stations[bus_stations['station_type_voie'] == closest_dest_station_type]

    if filtered_stations.empty:
        return "No station found."
    
    # Sort filtered stations by input distance in ascending order
    sorted_input_stations = filtered_stations.sort_values(by='distance_from_input')

    if sorted_input_stations.empty:
        return "No station found with the specified transport type."

    # Retrieve the nearest station coordinates, name, and type
    nearest_station = sorted_input_stations.iloc[0]
    nearest_station_coords = (nearest_station['station_coordinates'][0], nearest_station['station_coordinates'][1])
    nearest_station_type = nearest_station['station_type_voie']

    # Return the nearest station & closest destianation statio coordinates and type of line 
    return nearest_station_coords, closest_dest_station_coords, nearest_station_type

In [42]:
find_nearest_bus_station(48.846756, 2.316227, 48.922499, 2.3703)

((48.84723657995911, 2.3177792578249785),
 (48.92402658487458, 2.36886468338824),
 'VSU1')